# Mosaico 2021 C2

Script trabajado por Francisco Albornoz/ 
Objetivo:   - Mejorar la temporalidad de los mosaicos.
            - Usar la colección corregida.
            

2022-03-31

## Librerías

In [1]:
import ee 
import geemap
import geemap.chart as chart
import os
import folium
import xarray_leaflet
import rasterio 
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr
%matplotlib inline

In [2]:
Map = geemap.Map(center = [-45, -70], zoom = 4)
Map1 = geemap.Map(center = [-45, -70], zoom = 4)
Map2 = geemap.Map(center = [-45, -70], zoom = 4)
Map3 = geemap.Map(center = [-45, -70], zoom = 4)
Map4 = geemap.Map(center = [-45, -70], zoom = 4)
Map5 = geemap.Map(center = [-45, -70], zoom = 4)
Map6 = geemap.Map(center = [-45, -70], zoom = 4)
Map7 = geemap.Map(center = [-45, -70], zoom = 4)
Map8 = geemap.Map(center = [-45, -70], zoom = 4)
Map9 = geemap.Map(center = [-45, -70], zoom = 4)
Map10 = geemap.Map(center = [-45, -70], zoom = 4)

## Área de Estudio

In [3]:
work_dir = os.path.expanduser('../Data/')
patagonia = os.path.join(work_dir, 'patagonia_geo.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

## Funciones

### Visualizaciones

In [4]:
import geemap.colormaps as cm

In [5]:
color_ndvi = cm.palettes.ndvi
color_ndwi = cm.palettes.ndwi
color_terrain = cm.palettes.terrain
color_glaciar = cm.palettes.PuBuGn_r
color_nubes = cm.palettes.inferno

In [6]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [7]:
vizParams_m = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.6,
  'gamma': [1, 1, 1]
}

In [8]:
vis_nieve = {
  'bands': ['B6', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

### Factores

In [9]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMM'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [10]:
def ScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    return image.addBands(opticalBands, None, True)

In [11]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [12]:
def rename(image):
    bandas = (['B1', 'B2', 'B3', 'B4', 
                           'B5', 'B6', 'B7', 'SR_QA_AEROSOL', 
                           'B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 
                           'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 
                           'ST_URAD', 'QA_PIXEL', 'QA_RADSAT'])
    return image.rename(bandas)

### Índices

<<<< 1) Normalized vegetation Index >>>> 


NDVI = (NIR - RED)/(NIR + RED) | Rouse et al.

In [13]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

<<<< 2) Enhanced Vegetation Index >>>>


EVI = 2,5 * ((NIR - RED)/(NIR + 6 * RED - 7,5 * BLUE + 1))   |               Liu & Huete 

In [14]:
def addEVI(image):
    evi = image.expression(
        '2.5*((b5 - b4)/(b5 + 6*b4 - 7.5*b2 + 1))', {
            'b2':image.select('B2'),
            'b4':image.select('B4'),
            'b5':image.select('B5')
        }) 
    evi = evi.rename('EVI')
    return image.addBands(evi)

<<<< 3) Soil adjusted vegetation Index >>>>

SAVI = ((NIR - RED)/(NIR + RED + 0,5 )) * (1,5)    |   Huete

In [15]:
def addSAVI(image):
    savi = image.expression(
        '((b5 - b4)/(b5 + b4 + 0.5))*1.5', {
            'b4':image.select('B4'),
            'b5':image.select('B5')
        }) 
    savi= savi.rename('SAVI')
    return image.addBands(savi)

<<<< 4) Modified Soil adjusted vegetation Index >>>>


MSAVI2 = (2 * NIR + 1 - SQRT((2 * NIR +1)^2 - 8 * (NIR - RED)))/2   | Qi et al

In [16]:
def addMSAVI2(image):
    msavi = image.expression(
        '(2*b5 + 1 - (sqrt((2*b5 + 1)**2 - (8*(b5-b4)))))', {
            'b4':image.select('B4'),
            'b5':image.select('B5')
        })
    msavi= msavi.rename('MSAVI2')
    return image.addBands(msavi)

<<<< 5) Normalized water Index >>>>


NDWI = (NIR - SWIR) / (NIR + SWIR)  | Gao

In [17]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B5','B6']).rename('NDWI')
    return image.addBands(ndwi)

<<<< 6) Modified Normalized water Index >>>>


mNDWI = (GREEN - SWIR) / (GREEN + SWIR)  | Xu

In [18]:
def addmNDWI(image):
    mndwi = image.normalizedDifference(['B3','B6']).rename('mNDWI')
    return image.addBands(mndwi)

<<<< 7) Normalized water B Index >>>>


NDWBI = (GREEN - NIR) / (GREEN + NIR)  | McFeeters

In [19]:
def addNDWBI(image):
    ndwbi = image.normalizedDifference(['B3','B5']).rename('NDWBI')
    return image.addBands(ndwbi)

<<<< 8) Normalized Built Index >>>>


NDBI = (SWIR - NIR) / (SWIR + NIR)  | Zha et al.

In [20]:
def addNDBI(image):
    ndbi = image.normalizedDifference(['B6','B5']).rename('NDBI')
    return image.addBands(ndbi)

<<<< 9) Normalized Snow Index >>>>


NDSI = (GREEN - SWIR) / (GREEN + SWIR)  | 

In [21]:
def addNDSI(image):
    ndsi = image.normalizedDifference(['B3','B6']).rename('NDSI')
    return image.addBands(ndsi)

<<<< 10) Normalized Glaciar Index >>>>


NDGI = (GREEN - RED) / (GREEN + RED)  | 

In [22]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B3','B4']).rename('NDGI')
    return image.addBands(ndgi)

<<<< 11) Normalized Burn ratio Index >>>>


NBRI = (NIR - SWIR2) / (NIR + SWIR2)  |

In [23]:
def addNDBrI(image):
    ndbri = image.normalizedDifference(['B5','B7']).rename('NDBrI')
    return image.addBands(ndbri)

<<<< 12) Green Normalized vegetation Index >>>>


GNDVI = (NIR - GREEN) / (NIR + GREEN)  |

In [24]:
def addGNDVI(image):
    gndvi = image.normalizedDifference(['B5','B3']).rename('GNDVI')
    return image.addBands(gndvi)

### Clouds 

In [25]:
def addCloud(image):
    nube = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.addBands(ee.Image(nube.select(['cloud'])).rename('nube'))

In [26]:
def Cloud10(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(10))

In [27]:
def Cloud20(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20)) 

In [28]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [29]:
def Cloud40(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(40)) 

# Image Collections

## Mosaico

Verano 2022

In [30]:
start_date = '2021-11-01'
end_date = '2022-03-31'

l8_2021_2022_summercolection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)  

In [31]:
l8_2021_2022_summercolection = l8_2021_2022_summercolection \
    .map(addNDVI) \
    .map(addNDWI) \
    .map(addmNDWI) \
    .map(addNDWBI) \
    .map(addNDGI) \
    .map(addSAVI) \
    .map(addMSAVI2) \
    .map(addNDSI) \
    .map(addNDBI) \
    .map(addEVI) \
    .map(addNDBrI) \
    .map(addGNDVI) \
    .map(addDate) \
    .map(addCloud)

In [32]:
c2022_sn = l8_2021_2022_summercolection.map(Cloud10)
QM_NDVI_2022_sn = c2022_sn.qualityMosaic('NDVI')
QM_NDVI_2022 = l8_2021_2022_summercolection.qualityMosaic('NDVI')
QM_CLOUD_2022 = l8_2021_2022_summercolection.qualityMosaic('nube')
Med_NDVI_2022_sn = c2022_sn.median()

In [33]:
# QM_NDVI_2022_sn.bandNames().getInfo()

In [34]:
Map.addLayer(QM_NDVI_2022_sn, vizParams, 'QMsn de NDVI 2021-2022')
Map.addLayer(QM_NDVI_2022, vizParams, 'QM con nubes de NDVI 2021-2022')
Map.addLayer(QM_CLOUD_2022, vizParams, 'QM de nubes 2021-2022')
Map.addLayer(Med_NDVI_2022_sn, vizParams, 'Mediana sn 2021-2022')
Map

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

Verano 2021

In [35]:
start_date = '2020-11-01'
end_date = '2021-03-31'

l8_2020_2021_summercolection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)  

In [36]:
l8_2020_2021_summercolection = l8_2020_2021_summercolection \
    .map(addNDVI) \
    .map(addNDWI) \
    .map(addmNDWI) \
    .map(addNDWBI) \
    .map(addNDGI) \
    .map(addSAVI) \
    .map(addMSAVI2) \
    .map(addNDSI) \
    .map(addNDBI) \
    .map(addEVI) \
    .map(addNDBrI) \
    .map(addGNDVI) \
    .map(addDate) \
    .map(addCloud)

Verano 2020

In [37]:
start_date = '2019-11-01'
end_date = '2020-03-31'

l8_2019_2020_summercolection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)  

In [38]:
l8_2019_2020_summercolection = l8_2019_2020_summercolection \
    .map(addNDVI) \
    .map(addNDWI) \
    .map(addmNDWI) \
    .map(addNDWBI) \
    .map(addNDGI) \
    .map(addSAVI) \
    .map(addMSAVI2) \
    .map(addNDSI) \
    .map(addNDBI) \
    .map(addEVI) \
    .map(addNDBrI) \
    .map(addGNDVI) \
    .map(addDate) \
    .map(addCloud)

Verano 2019

In [39]:
start_date = '2018-11-01'
end_date = '2019-03-31'

l8_2018_2019_summercolection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date) 

In [40]:
l8_2018_2019_summercolection = l8_2018_2019_summercolection \
    .map(addNDVI) \
    .map(addNDWI) \
    .map(addmNDWI) \
    .map(addNDWBI) \
    .map(addNDGI) \
    .map(addSAVI) \
    .map(addMSAVI2) \
    .map(addNDSI) \
    .map(addNDBI) \
    .map(addEVI) \
    .map(addNDBrI) \
    .map(addGNDVI) \
    .map(addDate) \
    .map(addCloud)

## Colección completa

2 primeros años

In [41]:
filtersummer1 = ee.Filter.Or(
    ee.Filter.date('2021-11-01','2022-03-01'),
    ee.Filter.date('2020-11-01','2021-03-31'))

In [42]:
coleccion2022_2020 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterDate('2020-11-01', '2022-03-31') \
    .filter(filtersummer1) \
    .filterBounds(patagonia_grande)

In [43]:
l8_summer1= coleccion2022_2020 \
    .map(addNDVI) \
    .map(addNDWI) \
    .map(addmNDWI) \
    .map(addNDWBI) \
    .map(addNDGI) \
    .map(addSAVI) \
    .map(addMSAVI2) \
    .map(addNDSI) \
    .map(addNDBI) \
    .map(addEVI) \
    .map(addNDBrI) \
    .map(addGNDVI) \
    .map(addDate) \
    .map(addCloud)

In [44]:
l8_summer1_sn = l8_summer1.map(Cloud10)
QM_NDVI_summer1_sn = l8_summer1.qualityMosaic('NDVI')
QM_NDWI_summer1_sn = l8_summer1.qualityMosaic('NDWI')

3 años

In [45]:
filtersummer2 = ee.Filter.Or(
    ee.Filter.date('2021-11-01','2022-03-01'),
    ee.Filter.date('2020-11-01','2021-03-31'),
    ee.Filter.date('2019-11-01','2020-03-31')
)

In [46]:
coleccion2022_2019 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterDate('2019-11-01', '2022-03-31') \
    .filter(filtersummer2) \
    .filterBounds(patagonia_grande)

In [47]:
l8_summer2= coleccion2022_2019 \
    .map(addNDVI) \
    .map(addNDWI) \
    .map(addmNDWI) \
    .map(addNDWBI) \
    .map(addNDGI) \
    .map(addSAVI) \
    .map(addMSAVI2) \
    .map(addNDSI) \
    .map(addNDBI) \
    .map(addEVI) \
    .map(addNDBrI) \
    .map(addGNDVI) \
    .map(addDate) \
    .map(addCloud)

4 años

In [48]:
filtersummer4 = ee.Filter.Or(
    ee.Filter.date('2021-11-01','2022-03-01'),
    ee.Filter.date('2020-11-01','2021-03-31'),
    ee.Filter.date('2019-11-01','2020-03-31'),
    ee.Filter.date('2018-11-01','2019-03-31')
)

In [49]:
coleccion2022_2018 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterDate('2018-11-01', '2022-03-31') \
    .filter(filtersummer4) \
    .filterBounds(patagonia_grande)

In [50]:
l8_summer4= coleccion2022_2018 \
    .map(addNDVI) \
    .map(addNDWI) \
    .map(addmNDWI) \
    .map(addNDWBI) \
    .map(addNDGI) \
    .map(addSAVI) \
    .map(addMSAVI2) \
    .map(addNDSI) \
    .map(addNDBI) \
    .map(addEVI) \
    .map(addNDBrI) \
    .map(addGNDVI) \
    .map(addDate) \
    .map(addCloud)

In [51]:
cloudest = l8_summer4.qualityMosaic('nube')
l8_summer_sn = l8_summer4.map(Cloud10)
QM_NDWI_summer_sn = l8_summer_sn.qualityMosaic('NDWI')


5 Años

In [52]:
filtersummer5 = ee.Filter.Or(
    ee.Filter.date('2021-11-01','2022-03-01'),
    ee.Filter.date('2020-11-01','2021-03-31'),
    ee.Filter.date('2019-11-01','2020-03-31'),
    ee.Filter.date('2018-11-01','2019-03-31'),
    ee.Filter.date('2017-11-01','2018-03-31')
)

In [53]:
coleccion2022_2017_t = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterDate('2019-11-01', '2022-03-31') \
    .filterBounds(patagonia_grande)

In [54]:
coleccion2022_2017 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterDate('2017-11-01', '2022-03-31') \
    .filter(filtersummer5) \
    .filterBounds(patagonia_grande)

In [55]:
l8_summer5_t= coleccion2022_2017_t \
    .map(addNDVI) \
    .map(addNDWI) \
    .map(addmNDWI) \
    .map(addNDWBI) \
    .map(addNDGI) \
    .map(addSAVI) \
    .map(addMSAVI2) \
    .map(addNDSI) \
    .map(addNDBI) \
    .map(addEVI) \
    .map(addNDBrI) \
    .map(addGNDVI) \
    .map(addDate) \
    .map(addCloud)

In [56]:
l8_summer5= coleccion2022_2017 \
    .map(addNDVI) \
    .map(addNDWI) \
    .map(addmNDWI) \
    .map(addNDWBI) \
    .map(addNDGI) \
    .map(addSAVI) \
    .map(addMSAVI2) \
    .map(addNDSI) \
    .map(addNDBI) \
    .map(addEVI) \
    .map(addNDBrI) \
    .map(addGNDVI) \
    .map(addDate) \
    .map(addCloud)

In [57]:
l8_summer5_sn = l8_summer5.map(Cloud10)
l8_summer5_snt = l8_summer5_t.map(Cloud10)

3 Años verano más largo para llenar pixeles

In [58]:
filtersummer6 = ee.Filter.Or(
    ee.Filter.date('2021-09-01','2022-03-31'),
    ee.Filter.date('2020-09-01','2021-04-30'),
    ee.Filter.date('2019-09-01','2020-04-30'),
)

In [59]:
coleccion2022_2019_b = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterDate('2019-09-01', '2022-03-31') \
    .filter(filtersummer6) \
    .filterBounds(patagonia_grande)

In [60]:
l8_summer6= coleccion2022_2019_b \
    .map(addNDVI) \
    .map(addNDWI) \
    .map(addmNDWI) \
    .map(addNDWBI) \
    .map(addNDGI) \
    .map(addSAVI) \
    .map(addMSAVI2) \
    .map(addNDSI) \
    .map(addNDBI) \
    .map(addEVI) \
    .map(addNDBrI) \
    .map(addGNDVI) \
    .map(addDate) \
    .map(addCloud)

## Generación Mosaico QM

Tier 1

In [61]:
c2022_sn = l8_2021_2022_summercolection.map(Cloud10)
c2021_sn = l8_2020_2021_summercolection.map(Cloud10)
c2020_sn = l8_2019_2020_summercolection.map(Cloud10)
c2019_sn = l8_2018_2019_summercolection.map(Cloud10)

In [62]:
QM_NDVI_2022 = l8_2021_2022_summercolection.qualityMosaic('NDVI')
QM_NDVI_2021 = l8_2020_2021_summercolection.qualityMosaic('NDVI')
QM_NDVI_2020 = l8_2019_2020_summercolection.qualityMosaic('NDVI')
QM_NDVI_2019 = l8_2018_2019_summercolection.qualityMosaic('NDVI')

In [63]:
QM_NDVI_2022_sn = c2022_sn.qualityMosaic('NDVI')
QM_NDVI_2021_sn = c2021_sn.qualityMosaic('NDVI')
QM_NDVI_2020_sn = c2020_sn.qualityMosaic('NDVI')
QM_NDVI_2019_sn = c2019_sn.qualityMosaic('NDVI')

### Llenado de píxeles con nubes

In [64]:
cloudest_landsat = cloudest.set('SENSOR_ID', 'OLI_TIRS')
cloud_max = ee.Algorithms.Landsat.simpleCloudScore(cloudest_landsat)
cloud_max  = cloud_max .select('cloud')
palette_nubes = ['#1a9850', '#d73027']

In [65]:
resultados= [cloud_max , cloudest]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes 2021',
    'QM NDVI 2021']

In [66]:
# geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
#                    ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [67]:
def llenado(image):
    return image.where(cloud_max.gt(10),QM_NDVI_2022)

In [68]:
img1 = llenado(cloudest)

In [69]:
img1_landsat = img1.set('SENSOR_ID', 'OLI_TIRS')
cloud_img1 = ee.Algorithms.Landsat.simpleCloudScore(img1_landsat)
cloud_img1 = cloud_img1.select('cloud')

In [70]:
resultados= [cloud_img1, img1]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 1',
    'Mosaico 1']

In [71]:
# geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
#                    ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [72]:
def llenado2(image):
    return image.where(cloud_img1.gt(10), QM_NDVI_2021)

In [73]:
img2 = llenado2(img1)

In [74]:
img2_landsat = img2.set('SENSOR_ID', 'OLI_TIRS')
cloud_img2 = ee.Algorithms.Landsat.simpleCloudScore(img2_landsat)
cloud_img2 = cloud_img2.select('cloud')

In [75]:
resultados= [cloud_img2, img2_landsat]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 2',
    'Mosaico 2']

In [76]:
# geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
#                    ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [77]:
def llenado3(image):
    return image.where(cloud_img2.gt(10), QM_NDVI_2020)

In [78]:
img3 = llenado3(img2)

In [79]:
img3_landsat = img3.set('SENSOR_ID', 'OLI_TIRS')
cloud_img3 = ee.Algorithms.Landsat.simpleCloudScore(img3_landsat)
cloud_img3 = cloud_img3.select('cloud')

In [80]:
resultados= [cloud_img3, img3_landsat]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 3',
    'Mosaico 3']

In [81]:
# geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
#                    ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [82]:
def llenado4(image):
    return image.where(cloud_img3.gt(10), QM_NDVI_2019)

In [83]:
img4 = llenado4(img3)

In [84]:
img4_landsat = img4.set('SENSOR_ID', 'OLI_TIRS')
cloud_img4 = ee.Algorithms.Landsat.simpleCloudScore(img4_landsat)
cloud_img4 = cloud_img4.select('cloud')

In [85]:
resultados= [cloud_img4, img4_landsat]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 4',
    'Mosaico 4']

In [86]:
# geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
#                    ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [87]:
# hansenImage = ee.Image('UMD/hansen/global_forest_change_2020_v1_8')
# datamask = hansenImage.select('datamask')

In [88]:
# def llenado5(image):
#     return image.where(datamask.eq(2), QM_NDWI_summer_sn)

In [89]:
def llenado5(image):
    return image.where(cloud_img4.gt(10), l8_summer5_sn.median())

In [90]:
img5 = llenado5(img4)

In [91]:
img5_landsat = img5.set('SENSOR_ID', 'OLI_TIRS')
cloud_img5 = ee.Algorithms.Landsat.simpleCloudScore(img5_landsat)
cloud_img5 = cloud_img5.select('cloud')

In [92]:
resultados= [img4, img5]
labels = [
    'Mosaico 4',
    'Mosaico 5']
vis_params3=  [
     vizParams, 
    vizParams]

In [93]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params3, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [94]:
resultados= [cloud_img5, img5]
labels = [
    'Nubes',
    'Mosaico 5']
vis_params3=  [
     {'palette':palette_nubes}, 
    vizParams]

In [95]:
# geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
#                    ee_objects=resultados, vis_params=vis_params3, labels=labels, label_position="topright")

In [96]:
def llenado6(image):
    return image.where(cloud_img5.gt(10),l8_summer6.qualityMosaic('NDVI'))

In [97]:
img6 = llenado6(img5)

In [98]:
img6_landsat = img6.set('SENSOR_ID', 'OLI_TIRS')
cloud_img6 = ee.Algorithms.Landsat.simpleCloudScore(img6_landsat)
cloud_img6 = cloud_img6.select('cloud')

In [99]:
resultados= [img5, img6]
labels = [
    'Mosaico 5',
    'Mosaico 6']
vis_params3=  [
     vizParams, 
    vizParams]

In [100]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params3, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [101]:
resultados= [cloud_img6, img6]
labels = [
    'Nubes 6',
    'Mosaico 6']
vis_params3=  [
     {'palette':palette_nubes}, 
    vizParams]

In [102]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params3, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [103]:
hansenImage = ee.Image('UMD/hansen/global_forest_change_2015')
datamask = hansenImage.select('datamask')

In [104]:
def llenado7(image):
    return image.where(datamask.eq(2),l8_summer5_snt.qualityMosaic('NDWI'))

In [105]:
img7 = llenado7(img6)

In [106]:
resultados= [img6, img7]
labels = [
    'Mosaico 6',
    'Mosaico 7']
vis_params3=  [
     vizParams, 
    vizParams]

In [107]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params3, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [108]:
img7_landsat = img7.set('SENSOR_ID', 'OLI_TIRS')
cloud_img7 = ee.Algorithms.Landsat.simpleCloudScore(img7_landsat)
cloud_img7 = cloud_img7.select('cloud')

In [109]:
def llenado8(image):
    return image.where(cloud_img7.gt(10),l8_summer5_sn.median())

In [110]:
img8 = llenado8(img7)

In [111]:
resultados= [img7, img8]
labels = [
    'Mosaico 7',
    'Mosaico 8']
vis_params3=  [
     vizParams, 
    vizParams]

In [112]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params3, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [113]:
Map2.addLayer(cloud_img7,{'palette':palette_nubes},'Nubes 6')
Map2.addLayer(l8_summer6.qualityMosaic('NDVI'), vizParams, 'QM NDVI CLC')
Map2.addLayer(img8, vizParams, 'Mosaico Final')
Map2

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Descarga Mosaico

In [114]:
geometry_exp = ee.Geometry.Polygon(
        [[[-72.56281773604495, -33.81999514196144],
          [-72.96381871260745, -35.294382843203024],
          [-73.27692906416995, -35.25850698608905],
          [-74.15583531416995, -38.13757615203617],
          [-73.96357457198245, -38.16349478321529],
          [-74.41603250978783, -39.59214901926122],
          [-73.71840067385033, -39.70212049569608],
          [-74.48057965994737, -41.961233926702086],
          [-76.23199334221133, -46.599308425949715],
          [-75.71838250236758, -46.6705186900265],
          [-76.31583416780423, -48.08335993023956],
          [-75.6739504332226, -48.17601084778724],
          [-76.1812028986966, -49.3089458997328],
          [-75.8516130549466, -49.35190435466257],
          [-75.88182545729035, -49.42252520588146],
          [-75.62776661939972, -49.4593908439046],
          [-75.87838309772656, -50.0286830263256],
          [-75.4746355391328, -50.08951678614621],
          [-75.54325576346766, -50.23909657453096],
          [-75.77780697097086, -50.76007049594944],
          [-75.16119930495523, -50.84458853545005],
          [-75.38504574050211, -51.344039508567974],
          [-75.5541538866841, -51.71140521484993],
          [-75.18336531246536, -51.75648162278235],
          [-75.47788882049618, -52.3659753295982],
          [-74.75894718310815, -52.46884703800526],
          [-75.08565455837835, -53.06208069103826],
          [-74.55801479059262, -53.126414996444055],
          [-74.81690883977036, -53.6371915734125],
          [-73.44408000172288, -53.83741753537409],
          [-73.50862467945726, -53.95798563315933],
          [-73.45231974781663, -53.967680681913386],
          [-73.53197062672288, -54.11766576333605],
          [-73.77219189508037, -54.57485104154676],
          [-72.34049166499341, -54.77775087176573],
          [-72.47759402717024, -55.03231503944354],
          [-71.05285526571967, -55.24340674895388],
          [-71.08306766806342, -55.29113504937561],
          [-69.81960741798575, -55.47109132991434],
          [-69.85119311134513, -55.51932330880519],
          [-69.68227831642325, -55.544194128455445],
          [-69.79214159767325, -55.75341614311607],
          [-66.85410518726452, -56.21137383375629],
          [-66.30968752348785, -55.25322454720416],
          [-65.44176760161285, -55.378260612328376],
          [-65.29222191220039, -55.084940556988364],
          [-63.8145607793879, -55.29503930747886],
          [-63.4410256231379, -54.559610949790994],
          [-66.01731956845039, -54.18844646199444],
          [-65.92942894345039, -53.97576673331468],
          [-67.35765159970039, -53.77173637481835],
          [-66.83490788218774, -52.56216102836118],
          [-68.29059635875024, -52.351280557744616],
          [-67.13664467133525, -49.5665018755825],
          [-65.6891959408665, -49.77447361491692],
          [-64.6040407061826, -46.886773053508534],
          [-66.07346209290135, -46.670468623465915],
          [-65.60651222417681, -45.29237148760689],
          [-64.13159767339556, -45.51413819831776],
          [-63.11762183127383, -42.44210625114028],
          [-61.64820044455508, -42.65254570178212],
          [-60.691700635610566, -39.58656313087865],
          [-60.612049756704316, -39.597145764356114],
          [-60.214053037778356, -38.26345264755797],
          [-62.338430585556544, -37.86345563921212],
          [-62.431814374619044, -38.13616213643378],
          [-63.880475463562526, -37.8656239671591],
          [-63.965619506531276, -38.12752037872568],
          [-65.43863114657343, -37.86678770192114],
          [-65.52652177157343, -38.12867996539086],
          [-66.31753739657343, -37.98811028708228],
          [-65.98609312949165, -36.82548737245019],
          [-68.06866521074468, -36.43149799324939],
          [-68.16204899980718, -36.70723234878123],
          [-69.00250310136968, -36.55294237984414],
          [-68.66838285254276, -35.38834808765389],
          [-70.73190167990279, -35.008946233837705],
          [-70.82528546896529, -35.29190694633577],
          [-71.68771222677779, -35.11685679848462],
          [-71.37460187521529, -33.95679291780399],
          [-72.53365949240279, -33.733238932504804]]])

In [115]:
mosaico_final= img6.select('B1','B2','B3','B4','B5','B6','B7')


In [116]:
B1 = img6.select('B1')
B2 = img6.select('B2')
B3 = img6.select('B3')
B4 = img6.select('B4')
B5 = img6.select('B5')
B6 = img6.select('B6')
B7 = img6.select('B7')

In [117]:
task_mosaico_B1 = ee.batch.Export.image.toDrive(**{
    'image': B1,
    'description':'B1',
    'folder':'Earth Engine',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})
task_mosaico_B2 = ee.batch.Export.image.toDrive(**{
    'image': B2,
    'description':'B2',
    'folder':'Earth Engine',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})
task_mosaico_B3 = ee.batch.Export.image.toDrive(**{
    'image': B3,
    'description':'B3',
    'folder':'Earth Engine',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})
task_mosaico_B4 = ee.batch.Export.image.toDrive(**{
    'image': B4,
    'description':'B4',
    'folder':'Earth Engine',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})
task_mosaico_B5 = ee.batch.Export.image.toDrive(**{
    'image': B5,
    'description':'B5',
    'folder':'Earth Engine',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})
task_mosaico_B6 = ee.batch.Export.image.toDrive(**{
    'image': B6,
    'description':'B6',
    'folder':'Earth Engine',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})
task_mosaico_B7 = ee.batch.Export.image.toDrive(**{
    'image': B7,
    'description':'B7',
    'folder':'Earth Engine',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})

In [118]:
# task_mosaico_B1.start()
# task_mosaico_B2.start()
# task_mosaico_B3.start()
# task_mosaico_B4.start()
# task_mosaico_B5.start()
# task_mosaico_B6.start()
# task_mosaico_B7.start()


## Taseeled cap

In [119]:
tasse_img6 = img6.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])
arrayimage1dm = tasse_img6.toArray()
arrayimage2dm = arrayimage1dm.toArray(1)

In [120]:
coef = ee.Array([
    [0.3029, 0.2786, 0.4733, 0.5599, 0.508, 0.1872], 
    [-0.2941, -0.243, -0.5424, 0.7276, 0.0713, -0.1608],
    [0.1511, 0.1973, 0.3283, 0.3407, -0.7117, -0.4559],
    [-0.8239, 0.0849, 0.4396, -0.058, 0.2013, -0.2773],
    [-0.3294, 0.0557, 0.1056, 0.1855, -0.4349, 0.8085],
    [0.1079, -0.9023, 0.4119, 0.0575, -0.0259, 0.0252]
])

In [121]:
componentes = ee.Image(coef)\
                .matrixMultiply(arrayimage2dm) \
                .arrayProject([0])\
                .arrayFlatten(
                [['brightness','greenness','wetness','fourth','fifth','sixth']])

In [122]:
vizParamstess = {
  'bands': ['brightness', 'greenness', 'wetness'],
    'min': -0.1,
    'max': [0.6, 0.2, 0.2]
}

In [123]:
Map3.addLayer(componentes,vizParamstess, 'Tesseled cap')
Map3

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [124]:
tasseled = componentes.select('brightness','greenness','wetness')

## Texturas

TM

In [125]:
text_qm = img6.select('B1','B2','B3','B4','B5','B6','B7')
text_qm_array = text_qm.toArray()
text_qm_array = text_qm_array.toArray(1)

In [126]:
coef100 = ee.Array([
    [100,0,0,0,0,0,0],
    [0,100,0,0,0,0,0],
    [0,0,100,0,0,0,0],
    [0,0,0,100,0,0,0],
    [0,0,0,0,100,0,0],
    [0,0,0,0,0,100,0],
    [0,0,0,0,0,0,100],
    ])

In [127]:
factor_qm = ee.Image(coef100)\
                .matrixMultiply(text_qm_array) \
                .arrayProject([0])\
                .arrayFlatten(
                [['B1','B2','B3','B4','B5','B6','B7']])

In [128]:
testint_txt_qm = factor_qm.int()

In [129]:
txt_qm = testint_txt_qm.glcmTexture(3)

NDVI

In [130]:
ndvi_txt_array = img6.select('NDVI')
ndvi_txt_array = ndvi_txt_array.toArray()
ndvi_txt_array = ndvi_txt_array.toArray(1)

In [131]:
coef2 = ee.Array([[100]])

In [132]:
factor_ndvi = ee.Image(coef2)\
                .matrixMultiply(ndvi_txt_array) \
                .arrayProject([0])\
                .arrayFlatten(
                [['NDVI']])

In [133]:
ndvi_txt = factor_ndvi.int()

In [134]:
nvdi_texturas = ndvi_txt.glcmTexture(3)

In [135]:
texturas_ndvi = nvdi_texturas
texturas_ndvi = texturas_ndvi.toArray()
texturas_ndvi = texturas_ndvi.toArray(1)

In [136]:
coef3 = ee.Array([
    [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0.01,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
    ])

In [137]:
factor2 = ee.Image(coef3)\
                .matrixMultiply(texturas_ndvi) \
                .arrayProject([0])\
                .arrayFlatten(
                [['NDVI_asm',
 'NDVI_contrast',
 'NDVI_corr',
 'NDVI_var',
 'NDVI_idm',
 'NDVI_savg',
 'NDVI_svar',
 'NDVI_sent',
 'NDVI_ent',
 'NDVI_dvar',
 'NDVI_dent',
 'NDVI_imcorr1',
 'NDVI_imcorr2',
 'NDVI_maxcorr',
 'NDVI_diss',
 'NDVI_inertia',
 'NDVI_shade',
 'NDVI_prom']])

EVI

In [138]:
EVI_txt_array = img6.select('EVI')
EVI_txt_array = ndvi_txt_array.toArray()
EVI_txt_array = ndvi_txt_array.toArray(1)

In [139]:
coef2 = ee.Array([[100]])

In [140]:
factor_evi = ee.Image(coef2)\
                .matrixMultiply(EVI_txt_array) \
                .arrayProject([0])\
                .arrayFlatten(
                [['EVI']])

In [141]:
factor_evi_txt = factor_evi.int()

In [142]:
txt_evi = factor_evi_txt.glcmTexture(3)

In [143]:
texturas_evi = txt_evi
texturas_evi = texturas_evi.toArray()
texturas_evi = texturas_evi.toArray(1)

In [144]:
coef3 = ee.Array([
    [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0.01,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
    ])

In [145]:
factor3 = ee.Image(coef3)\
                .matrixMultiply(texturas_evi) \
                .arrayProject([0])\
                .arrayFlatten(
                [['EVI_asm',
 'EVI_contrast',
 'EVI_corr',
 'EVI_var',
 'EVI_idm',
 'EVI_savg',
 'EVI_svar',
 'EVI_sent',
 'EVI_ent',
 'EVI_dvar',
 'EVI_dent',
 'EVI_imcorr1',
 'EVI_imcorr2',
 'EVI_maxcorr',
 'EVI_diss',
 'EVI_inertia',
 'EVI_shade',
 'EVI_prom']])

## DEM

In [146]:
dem = ee.Image('USGS/SRTMGL1_003').clip(patagonia_grande)

In [147]:
elevation = dem.select('elevation')

In [148]:
slope = ee.Terrain.slope(elevation)

In [149]:
aspect = ee.Terrain.aspect(dem)

In [150]:
dem_data = elevation.addBands([slope,aspect])

## NDVI Index series

Primavera

In [151]:
filterspring = ee.Filter.Or(
    ee.Filter.date('2021-09-15','2021-10-31'),
    ee.Filter.date('2020-09-15','2020-10-31'),
    ee.Filter.date('2019-09-15','2019-10-31'),
    ee.Filter.date('2018-09-15','2018-10-31')
)

In [152]:
start_date = '2018-09-15'
end_date = '2021-10-31'

l8_spring = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filter(filterspring) \
    .filterDate(start_date, end_date)

In [153]:
NDVI_spring = l8_spring.map(addNDVI)
NDVI_spring = NDVI_spring.median()
NDVI_spring = NDVI_spring.select('NDVI').rename('NDVI_spring')

Invierno

In [154]:
filterwinter = ee.Filter.Or(
    ee.Filter.date('2021-07-01','2021-09-14'),
    ee.Filter.date('2020-07-01','2020-09-14'),
    ee.Filter.date('2019-07-01','2019-09-14'),
    ee.Filter.date('2018-07-01','2018-09-14')
)

In [155]:
start_date = '2018-07-01'
end_date = '2021-09-14'

l8_winter = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filter(filterwinter) \
    .filterDate(start_date, end_date)

In [156]:
NDVI_winter = l8_winter.map(addNDVI)
NDVI_winter = NDVI_winter.median()
NDVI_winter = NDVI_winter.select('NDVI').rename('NDVI_winter')

Otoño

In [157]:
filterautumn = ee.Filter.Or(
    ee.Filter.date('2021-04-01','2021-06-30'),
    ee.Filter.date('2020-04-01','2020-06-30'),
    ee.Filter.date('2019-04-01','2019-06-30'),
    ee.Filter.date('2018-04-01','2018-06-30')
)

In [158]:
start_date = '2018-04-01'
end_date = '2021-06-30'

l8_autumn = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filter(filterautumn) \
    .filterDate(start_date, end_date)

In [159]:
NDVI_autumn = l8_autumn.map(addNDVI)
NDVI_autumn = NDVI_autumn.median()
NDVI_autumn = NDVI_autumn.select('NDVI').rename('NDVI_autumn')

## Long/Lat

In [160]:
long_lat = img6.pixelLonLat()

In [161]:
long_lat.bandNames().getInfo()

['longitude', 'latitude']

In [162]:
# utm = (long_lat.select('longitude')).add(180).divide(6).int()

In [163]:
# Map4.addLayer(long_lat.select('longitude'),{},'longituda')
# Map4.addLayer(utm,{min:0, max: 60},'longitud')
# Map4

# Clasificación

In [164]:
img6.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'QA_PIXEL',
 'QA_RADSAT',
 'SAA',
 'SZA',
 'VAA',
 'VZA',
 'NDVI',
 'NDWI',
 'mNDWI',
 'NDWBI',
 'NDGI',
 'SAVI',
 'MSAVI2',
 'NDSI',
 'NDBI',
 'EVI',
 'NDBrI',
 'GNDVI',
 'date',
 'nube']

In [165]:
img_final = img6.select('B1','B2','B3','B4','B5','B6','B7','NDVI','NDWI','mNDWI','NDWBI','NDGI','SAVI','MSAVI2','NDSI','NDBI','EVI','NDBrI','GNDVI')
img_final = img_final.addBands(tasseled).addBands(dem_data).addBands(NDVI_spring).addBands(NDVI_winter).addBands(NDVI_autumn).addBands(factor2).addBands(factor3) 

In [166]:
img_final.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'NDVI',
 'NDWI',
 'mNDWI',
 'NDWBI',
 'NDGI',
 'SAVI',
 'MSAVI2',
 'NDSI',
 'NDBI',
 'EVI',
 'NDBrI',
 'GNDVI',
 'brightness',
 'greenness',
 'wetness',
 'elevation',
 'slope',
 'aspect',
 'NDVI_spring',
 'NDVI_winter',
 'NDVI_autumn',
 'NDVI_asm',
 'NDVI_contrast',
 'NDVI_corr',
 'NDVI_var',
 'NDVI_idm',
 'NDVI_savg',
 'NDVI_svar',
 'NDVI_sent',
 'NDVI_ent',
 'NDVI_dvar',
 'NDVI_dent',
 'NDVI_imcorr1',
 'NDVI_imcorr2',
 'NDVI_maxcorr',
 'NDVI_diss',
 'NDVI_inertia',
 'NDVI_shade',
 'NDVI_prom',
 'EVI_asm',
 'EVI_contrast',
 'EVI_corr',
 'EVI_var',
 'EVI_idm',
 'EVI_savg',
 'EVI_svar',
 'EVI_sent',
 'EVI_ent',
 'EVI_dvar',
 'EVI_dent',
 'EVI_imcorr1',
 'EVI_imcorr2',
 'EVI_maxcorr',
 'EVI_diss',
 'EVI_inertia',
 'EVI_shade',
 'EVI_prom']

In [167]:
bands = ['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'NDVI',
 'NDWI',
 'mNDWI',
 'NDWBI',
 'NDGI',
 'SAVI',
 'MSAVI2',
 'NDSI',
 'NDBI',
 'EVI',
 'NDBrI',
 'GNDVI',
 'brightness',
 'greenness',
 'wetness',
 'elevation',
 'slope',
 'aspect',
 'NDVI_spring',
 'NDVI_winter',
 'NDVI_autumn',
 'NDVI_asm',
 'NDVI_contrast',
 'NDVI_corr',
 'NDVI_var',
 'NDVI_idm',
 'NDVI_savg',
 'NDVI_svar',
 'NDVI_sent',
 'NDVI_ent',
 'NDVI_dvar',
 'NDVI_dent',
 'NDVI_imcorr1',
 'NDVI_imcorr2',
 'NDVI_maxcorr',
 'NDVI_diss',
 'NDVI_inertia',
 'NDVI_shade',
 'NDVI_prom',
 'EVI_asm',
 'EVI_contrast',
 'EVI_corr',
 'EVI_var',
 'EVI_idm',
 'EVI_savg',
 'EVI_svar',
 'EVI_sent',
 'EVI_ent',
 'EVI_dvar',
 'EVI_dent',
 'EVI_imcorr1',
 'EVI_imcorr2',
 'EVI_maxcorr',
 'EVI_diss',
 'EVI_inertia',
 'EVI_shade',
 'EVI_prom']

In [203]:
test1 = os.path.join(work_dir,'roi_4.shp')
test1 = geemap.shp_to_ee(test1)

In [185]:
# random = os.path.join(work_dir,'random_points.shp')
# random = geemap.shp_to_ee(random)

In [186]:
# points= img_final.sampleRegions(**{
#     'collection': random,
#     'properties': ['ID'],
#     'scale': 10,
#     'tileScale': 3,
# })

In [187]:
# Map6.addLayer(points, {},'muestras')
# Map6

In [188]:
# tablas_final = ee.batch.Export.table.toDrive(**{
#   'collection': points,
#   'description': 'datos',
#   'fileFormat': 'CSV'
# })

In [189]:
# tablas_final.start()

In [204]:
data_QM = img_final.sampleRegions(**{
    'collection': test1,
    'properties': ['C_ID'],
    'scale': 30,
    'tileScale': 2,
})

In [191]:
tablas = ee.batch.Export.table.toDrive(**{
  'collection': data_QM,
  'description': 'TM_summer_QM',
  'fileFormat': 'CSV'
})

In [192]:
# tablas.start()

## Muestra y validación

In [205]:
sample1 = data_QM.randomColumn()
split = 0.7

In [206]:
training1 = sample1.filter(ee.Filter.lt('random', split))
validation1= sample1.filter(ee.Filter.gte('random', split))

In [207]:
label = 'IDH'
label1 = 'ID'
label_train ='C_ID'

In [208]:
classifier_QM = ee.Classifier.smileRandomForest(100).train(data_QM,label_train,bands)

In [209]:
result_QM = img_final.select(bands).classify(classifier_QM)

In [210]:
explained_QM = ee.Classifier.explain(classifier_QM)

In [211]:
explained_QM.getInfo()

EEException: Unable to transform geometry into projection <Projection>.

### Categorizando

In [ ]:
class_values2 = [1,2,3,4,5,6,7,8,9,10,12,13,15,16,19,20,23,30,31,32,33,34,35,36,39,40,47,48,49,60]

class_palette2 = ['d1fbff', # 1 Hielo
 'bfdadd', #2 lengua glaciar
 'b9bfbf', #3 glaciar cubierto
 'fe40fe', #4 nieve
 '8ac1ff', #5 fiordo glaciar
 '0c407b', #6 mar
 '2e8da0', #7 lago continental
 '16b2d1', #8 lago glaciar
 '27e1d8', #9 río
 'eeb514', #10 suelo desnudo
#  '927012', #11 cárcavas erosión 
 'e98c0f', #12 suelo río
 'f7e8bd', #13 suelo duna/arenal
#  '5b5b5b', #14 caminos
 'e57104', #15 suelo desnudo / poca vegetación
 '5d1412', #16 tuberas / musgos
#  '5d1412', #17 mallín                 
 '161163', #19 áreas inundables
 'fff300', #20 plantación adulto
 'ff0000', #23 ciudad urbano
 'dee842', #30 praderas pastoreo             
 'ced118', #31 estepa abierta
 '7a7c0f', #32 estepa cerrada
 'c39700', #33 palizales
 'a1ff8a', #34 praderas
 '138665', #35 matorrales densos
 '1bbd8e', #36 matorrales abiertos
 '195a09', #39 primario adulto
 '27c41d', #40 secundario adulto
#  'b2ff00', #43 renoval inicial
#  'f36605', #45 degradado
 '7729db', #47 Primario adulto caduco
 '9e60ec', #48 Secundario adulto caduco
 'f36605', #49 achaparrados caduco
#  '7000ff', #57 Mixto Primario
#  'c598ff', #58 Mixto secundario
 '00ff87', #60 Coníferas nativas
]

In [ ]:
landcover_QM = result_QM.set('classification_class_values', class_values2)
landcover_QM = landcover_QM.set('classification_class_palette',class_palette2)

In [ ]:
Map5.addLayer(landcover_QM,{}, 'Land cover RF-QM' )
Map5

# Comparación de Metología

## Met. Andrés

In [ ]:
# img_ba= img_final.select('B4','B5','EVI','EVI_corr','EVI_imcorr1','NDVI_diss','NDVI_spring','NDWI','elevation','mNDWI','slope','EVI_ent').addBands(long_lat)

In [ ]:
# img_ba.bandNames().getInfo()

In [ ]:
# data_ba = img_ba.sampleRegions(**{
#     'collection': test1,
#     'properties': ['C_ID'],
#     'scale': 30,
#     'tileScale': 2,
# })

In [ ]:
# sample2 = data_ba.randomColumn()
# split = 0.7

In [ ]:
# training2 = sample2.filter(ee.Filter.lt('random', split))
# validation2= sample2.filter(ee.Filter.gte('random', split))

In [ ]:
# bands_ba = ['B4','B5','EVI','EVI_corr','EVI_imcorr1','NDVI_diss','NDVI_spring','NDWI','elevation','mNDWI','slope','EVI_ent','longitude', 'latitude']

In [ ]:
# classifier_ba = ee.Classifier.smileRandomForest(100).train(training2,label_train,bands_ba)

In [ ]:
# result_ba = img_ba.select(bands_ba).classify(classifier_ba)

In [ ]:
# explained_ba = ee.Classifier.explain(classifier_ba)

In [ ]:
# explained_ba.getInfo()

In [ ]:
# class_values2 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,23,30,31,32,33,34,35,36,39,40,43,45,47,48,57,58,60]

# class_palette2 = ['d1fbff', # 1 Hielo
#  'bfdadd', #2 lengua glaciar
#  'b9bfbf', #3 glaciar cubierto
#  'fe40fe', #4 nieve
#  '8ac1ff', #5 fiordo glaciar
#  '0c407b', #6 mar
#  '2e8da0', #7 lago continental
#  '16b2d1', #8 lago glaciar
#  '27e1d8', #9 río
#  'eeb514', #10 suelo desnudo
#  '927012', #11 cárcavas erosión 
#  'e98c0f', #12 suelo río
#  'f7e8bd', #13 suelo duna/arenal
#  '5b5b5b', #14 caminos
#  'e57104', #15 suelo desnudo / poca vegetación
#  '5d1412', #16 tuberas / musgos
#  '5d1412', #17 mallín                 
#  '161163', #19 áreas inundables
#  'fff300', #20 plantación adulto
#  'ff0000', #23 ciudad urbano
#  'dee842', #30 praderas pastoreo             
#  'ced118', #31 estepa abierta
#  '7a7c0f', #32 estepa cerrada
#  'c39700', #33 palizales
#  'a1ff8a', #34 preaderas
#  '138665', #35 matorrales densos
#  '1bbd8e', #36 matorrales abiertos
#  '195a09', #39 primario adulto
#  '27c41d', #40 secundario adulto
#  'b2ff00', #43 renoval inicial
#  'f36605', #45 degradado
#  '7729db', #47 Primario adulto caduco
#  '9e60ec', #48 Secundario adulto caduco
#  '7000ff', #57 Mixto Primario
#  'c598ff', #58 Mixto secundario
#  '00ff87', #60 Coníferas nativas
# ]

In [ ]:
# landcover_ba = result_ba.set('classification_class_values', class_values2)
# landcover_ba = landcover_ba.set('classification_class_palette',class_palette2)

In [ ]:
# Map7.addLayer(landcover_ba,{}, 'Land cover Ba' )
# Map7

## Met. RFE

### Primer

In [ ]:
img_rfe = img_final.select('elevation','NDVI_spring','NDVI_winter','NDVI_autumn','B1','NDGI','EVI_savg','B2','NDVI_savg','mNDWI','B4','B3','brightness',
                           'NDSI','slope','NDBI','NDWI','wetness','B7','NDBrI','greenness','B5','EVI','B6','NDWBI','GNDVI','NDVI','MSAVI2','SAVI')

In [ ]:
img_rfe.bandNames().getInfo()

In [ ]:
data_rfe = img_rfe.sampleRegions(**{
    'collection': test1,
    'properties': ['C_ID'],
    'scale': 30,
    'tileScale': 2,
})

In [ ]:
sample3 = data_rfe.randomColumn()
split = 0.7

In [ ]:
training3 = sample3.filter(ee.Filter.lt('random', split))
validation3= sample3.filter(ee.Filter.gte('random', split))

In [ ]:
bands_rfe = ['elevation','NDVI_spring','NDVI_winter','NDVI_autumn','B1','NDGI','EVI_savg','B2','NDVI_savg','mNDWI','B4','B3','brightness',
                           'NDSI','slope','NDBI','NDWI','wetness','B7','NDBrI','greenness','B5','EVI','B6','NDWBI','GNDVI','NDVI','MSAVI2','SAVI']

In [ ]:
classifier_rfe = ee.Classifier.smileRandomForest(100).train(training3,label_train,bands_rfe)

In [ ]:
result_rfe = img_rfe.select(bands_rfe).classify(classifier_rfe)

In [ ]:
explained_rfe = ee.Classifier.explain(classifier_rfe)

In [ ]:
explained_rfe.getInfo()

In [ ]:
landcover_rfe = result_rfe.set('classification_class_values', class_values2)
landcover_rfe = landcover_rfe.set('classification_class_palette',class_palette2)

In [ ]:
Map8.addLayer(landcover_rfe,{}, 'Land cover rfe' )
Map8

### Segundo

In [ ]:
img_rfe2 = img_final.select('elevation','NDVI_spring','NDVI_winter','NDVI_autumn','B1','NDGI','EVI_savg','B2','NDVI_savg','mNDWI','B4','B3','brightness',
                           'NDSI','slope','NDBI','NDWI','wetness','B7','NDBrI','greenness','B5','EVI')

In [ ]:
img_rfe2.bandNames().getInfo()

In [ ]:
data_rfe2 = img_rfe2.sampleRegions(**{
    'collection': test1,
    'properties': ['C_ID'],
    'scale': 30,
    'tileScale': 2,
})

In [ ]:
sample4 = data_rfe2.randomColumn()
split = 0.7

In [ ]:
training4 = sample4.filter(ee.Filter.lt('random', split))
validation4= sample4.filter(ee.Filter.gte('random', split))

In [ ]:
bands_rfe2 = ['elevation','NDVI_spring','NDVI_winter','NDVI_autumn','B1','NDGI','EVI_savg','B2','NDVI_savg','mNDWI','B4','B3','brightness',
                           'NDSI','slope','NDBI','NDWI','wetness','B7','NDBrI','greenness','B5','EVI']

In [ ]:
classifier_rfe2 = ee.Classifier.smileRandomForest(100).train(training4,label_train,bands_rfe2)

In [ ]:
result_rfe2 = img_rfe.select(bands_rfe2).classify(classifier_rfe2)

In [ ]:
explained_rfe2 = ee.Classifier.explain(classifier_rfe2)

In [ ]:
explained_rfe2.getInfo()

In [ ]:
landcover_rfe2 = result_rfe2.set('classification_class_values', class_values2)
landcover_rfe2 = landcover_rfe2.set('classification_class_palette',class_palette2)

In [ ]:
Map9.addLayer(landcover_rfe2,{}, 'Land cover rfe2' )
Map9

# Matrices de Confusión

Primer mosaico Bruto

In [ ]:
train_accuracy = classifier_QM.confusionMatrix()

In [ ]:
train_accuracy.getInfo()

In [ ]:
train_accuracy.accuracy().getInfo()

In [ ]:
train_accuracy.kappa().getInfo()

In [ ]:
train_accuracy.producersAccuracy().getInfo()

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

In [ ]:
validated = validation1.classify(classifier_QM)

In [ ]:
# validated.first().getInfo()

In [ ]:
test_accuracy = validated.errorMatrix('C_ID', 'classification')

In [ ]:
test_accuracy.getInfo()

In [ ]:
test_accuracy.accuracy().getInfo()

In [ ]:
test_accuracy.kappa().getInfo()

In [ ]:
test_accuracy.producersAccuracy().getInfo()

In [ ]:
test_accuracy.consumersAccuracy().getInfo()

Met Ba

In [ ]:
train_accuracy = classifier_ba.confusionMatrix()

In [ ]:
train_accuracy.getInfo()

In [ ]:
train_accuracy.accuracy().getInfo()

In [ ]:
train_accuracy.kappa().getInfo()

In [ ]:
train_accuracy.producersAccuracy().getInfo()

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

In [ ]:
validated = validation2.classify(classifier_ba)

In [ ]:
# validated.first().getInfo()

In [ ]:
test_accuracy = validated.errorMatrix('C_ID', 'classification')

In [ ]:
test_accuracy.getInfo()

In [ ]:
test_accuracy.accuracy().getInfo()

In [ ]:
test_accuracy.kappa().getInfo()

In [ ]:
test_accuracy.producersAccuracy().getInfo()

In [ ]:
test_accuracy.consumersAccuracy().getInfo()

Met. RFE1

In [ ]:
train_accuracy = classifier_rfe.confusionMatrix()

In [ ]:
train_accuracy.getInfo()

In [ ]:
train_accuracy.accuracy().getInfo()

In [ ]:
train_accuracy.kappa().getInfo()

In [ ]:
train_accuracy.producersAccuracy().getInfo()

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

In [ ]:
validated = validation3.classify(classifier_rfe)

In [ ]:
# validated.first().getInfo()

In [ ]:
test_accuracy = validated.errorMatrix('C_ID', 'classification')

In [ ]:
test_accuracy.getInfo()

In [ ]:
test_accuracy.accuracy().getInfo()

In [ ]:
test_accuracy.kappa().getInfo()

In [ ]:
test_accuracy.producersAccuracy().getInfo()

In [ ]:
test_accuracy.consumersAccuracy().getInfo()

Met. RFE2

In [ ]:
train_accuracy = classifier_rfe2.confusionMatrix()

In [ ]:
train_accuracy.getInfo()

In [ ]:
train_accuracy.accuracy().getInfo()

In [ ]:
train_accuracy.kappa().getInfo()

In [ ]:
train_accuracy.producersAccuracy().getInfo()

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

In [ ]:
validated = validation4.classify(classifier_rfe2)

In [ ]:
# validated.first().getInfo()

In [ ]:
test_accuracy = validated.errorMatrix('C_ID', 'classification')

In [ ]:
test_accuracy.getInfo()

In [ ]:
test_accuracy.accuracy().getInfo()

In [ ]:
test_accuracy.kappa().getInfo()

In [ ]:
test_accuracy.producersAccuracy().getInfo()

In [ ]:
test_accuracy.consumersAccuracy().getInfo()